- cancel transfer leanring

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

sys.path.append('../input/multilabelstraifier/')
sys.path.append('../input/lookahead/')
from ml_stratifiers import MultilabelStratifiedKFold
from lookahead import Lookahead
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
noncons_train_index = list(noncons_train_index) #+ original_remove_index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
non_targets = non_targets[non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
nontarget_dists.columns = ["target", "number"]
nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
print("first drop", len(drop_list1))
non_targets.drop(drop_list1, axis=1, inplace=True)
print("shape after 1st drop:", non_targets.shape)
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
print(len(non_target_feats))

first drop 71
shape after 1st drop: (21948, 332)
331


# Feature engineering 

In [7]:
# rank gauss
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal") #確かに正規分布っぽくなっている。
    
    vec_len = len(train[i].values)
    vec_len_test = len(test[i].values)
    raw_vec = train[i].values.reshape(vec_len, 1)
    ss.fit(raw_vec)

    train[i] = ss.transform(raw_vec).reshape(1, vec_len)[0]
    test[i] = ss.transform(test[i].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [8]:
num = 10
pca_c_cols = ["pca-c"+str(i+1) for i in range(num)]
pca = PCA(n_components=num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

num = 40
pca_g_cols = ["pca-g"+str(i+1) for i in range(num)]
pca = PCA(n_components=num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [9]:
X = train.iloc[:,4:].copy().values
select = VarianceThreshold(threshold=0.5)
X_new = select.fit_transform(X)
drop_feats = list(np.array(train.iloc[:,4:].columns)[select.get_support()==False])
len(drop_feats)

0

In [10]:
def fe(df):
    tmp = df.copy()
    #tmp['g_sum'] = tmp[g_feats].sum(axis = 1)
    #tmp['g_mean'] = tmp[g_feats].mean(axis = 1)
    #tmp['g_std'] = tmp[g_feats].std(axis = 1)
    #tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    #tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    #tmp['c_sum'] = tmp[c_feats].sum(axis = 1)
    #tmp['c_mean'] = tmp[c_feats].mean(axis = 1)
    #tmp['c_std'] = tmp[c_feats].std(axis = 1)
    #tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    #tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    #tmp['gc_sum'] = tmp[c_feats + g_feats].sum(axis = 1)
    #tmp['gc_mean'] = tmp[c_feats + g_feats].mean(axis = 1)
    #tmp['gc_std'] = tmp[c_feats + g_feats].std(axis = 1)
    #tmp['gc_kurt'] = tmp[c_feats + g_feats].kurtosis(axis = 1)
    #tmp['gc_skew'] = tmp[c_feats + g_feats].skew(axis = 1)
    #tmp.loc[:, 'cp_type'] = tmp.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    #tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    tmp = pd.get_dummies(tmp, columns=['cp_time','cp_dose'])
        
    tmp.drop(["cp_type", "sig_id"], axis=1, inplace=True)
    return tmp

train = fe(train).to_numpy()
test = fe(test).to_numpy()

print(train.shape, test.shape)

(21948, 927) (3982, 927)


In [11]:
#fn_train = train.copy().to_numpy()
#fn_test = test.copy().to_numpy()

#ss = preprocessing.RobustScaler()
#fn_train= ss.fit_transform(fn_train)
#fn_test = ss.transform(fn_test)

fn_non_targets = non_targets.drop("sig_id", axis=1).copy().to_numpy()
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

# modelling

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=42): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class MoaModel(nn.Module):
    def __init__(self, num_columns, last_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024,1024))
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, last_num))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

cuda


# modelling

In [13]:
batch_size = 128
n_folds=5
EARLY_STOPPING_STEPS = 25
train_epochs = 25

def modelling_torch(tr, target, te, sample_seed, init_num, last_num):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]
    
    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=224)

    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
            
        clf = MoaModel(init_num, last_num)
        loss_fn = torch.nn.BCEWithLogitsLoss() 

        optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
        #lookahead = Lookahead(optimizer, k=10, alpha=0.6) #lookahead
        #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e1, 
                                              max_lr=1e-2, epochs=25, steps_per_epoch=len(train_loader))
        
        clf.to(device)
        
        best_val_loss = np.inf
        stop_counts = 0
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)        
            
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)
                    
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')
            else:
                stop_counts += 1
        
            #if stop_counts >= EARLY_STOPPING_STEPS: 
            #    break
        pred_model = MoaModel(init_num, last_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))         
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch,) in enumerate(test_loader): 
            y_pred = pred_model(x_batch).sigmoid().detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
        pred_value += test_preds / n_folds
        # ------------------------------
        
    print("Seed {}".format(seed_))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    return oof, oof_targets, pred_value

In [14]:
seeds = [0,1,2,3]
target_oof = np.zeros([len(train),fn_targets.shape[1]])
target_pred = np.zeros([len(test),fn_targets.shape[1]])

nontarget_oof = np.zeros([len(train),fn_non_targets.shape[1]])
nontarget_pred = np.zeros([len(test),fn_non_targets.shape[1]])

for seed_ in seeds:
    oof, oof_targets, pytorch_pred = modelling_torch(train, fn_targets, test, seed_, train.shape[1], fn_targets.shape[1])
    target_oof += oof / len(seeds)
    target_pred += pytorch_pred / len(seeds)
print("Total log loss in targets: {}".format(mean_log_loss(oof_targets, target_oof)))

#for seed_ in seeds:
#    oof, oof_targets, pytorch_pred = modelling_torch(train, fn_non_targets, test, seed_, train.shape[1], fn_non_targets.shape[1])
#    nontarget_oof += oof / len(seeds)
#    nontarget_pred += pytorch_pred / len(seeds)
#print("Total log loss in Non targets: {}".format(mean_log_loss(oof_targets, nontarget_oof)))

Fold 1
Best model: Epoch 1 	 loss=0.410298 	 val_loss=0.080772 	 time=1.76s
Best model: Epoch 2 	 loss=0.045649 	 val_loss=0.027791 	 time=0.85s
Best model: Epoch 3 	 loss=0.026340 	 val_loss=0.022323 	 time=0.89s
Best model: Epoch 4 	 loss=0.022533 	 val_loss=0.020522 	 time=0.97s
Best model: Epoch 5 	 loss=0.020916 	 val_loss=0.019512 	 time=1.47s
Best model: Epoch 6 	 loss=0.020083 	 val_loss=0.018881 	 time=1.06s
Best model: Epoch 7 	 loss=0.019489 	 val_loss=0.018548 	 time=0.86s
Best model: Epoch 8 	 loss=0.018890 	 val_loss=0.018123 	 time=0.82s
Best model: Epoch 9 	 loss=0.018482 	 val_loss=0.017825 	 time=0.88s
Best model: Epoch 10 	 loss=0.017990 	 val_loss=0.017534 	 time=0.81s
Best model: Epoch 11 	 loss=0.017523 	 val_loss=0.017194 	 time=0.80s
Best model: Epoch 12 	 loss=0.017161 	 val_loss=0.017091 	 time=0.80s
Best model: Epoch 13 	 loss=0.016895 	 val_loss=0.016929 	 time=0.82s
Best model: Epoch 14 	 loss=0.016614 	 val_loss=0.016768 	 time=0.79s
Best model: Epoch 15 	

In [15]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)

print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.014738127167532242


In [16]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)